In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from warnings import simplefilter
warnings.filterwarnings("ignore", category =UserWarning,)

import gc
import pickle
from collections import OrderedDict
from pyproj import Transformer
import numpy as np
import pandas as pd
import iris
import glob
import sys
import os
import cartopy.crs as ccrs
import itertools
from scipy import spatial
import numpy.ma as ma
import tilemapbase
from math import cos, radians
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from pyproj import Proj, transform
import time

from Identify_Events_Functions import *
from Prepare_Data_Functions import *

pd.set_option('display.float_format', '{:.3f}'.format)
warnings.filterwarnings("ignore", category=UserWarning)

yrs_range= '2060_2081'
em = 'bb198'

# Get Tb0 values at each gauge
tbo_vals = pd.read_csv('/nfs/a319/gy17m2a/PhD/datadir/RainGauge/interarrival_thresholds_CDD_noMissing.txt')
# Read in a sample cube for finding the location of gauge in grid
sample_cube = iris.load(f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng/{yrs_range}/{em}/bng_{em}a.pr206101.nc')[0][1,:,:]

for yr in range(2061,2081):
    print(yr)
    ######################################################
    ### Get all the data for one year, into one cube
    # (if it already exists in a pickle file, then load it from there)
    ######################################################
    general_filename = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng/{yrs_range}/{em}/bng_{em}a.pr{yr}*'
    pickle_file_filepath = f"/nfs/a319/gy17m2a/PhD/datadir/cache/UKCP18_30mins_{em}//WholeYear/cube_{yr}.pkl"

    if os.path.exists(pickle_file_filepath):
        print("Pickle file exists, so loading that")
        full_year_cube = load_cube_from_picklefile(pickle_file_filepath)
    else:
        print("Pickle file doesnt exist, so creating and then saving that")

        ### Get the data filepaths
        print(f"Loading data for year {yr}")

        # Create cube list
        cubes = load_files_to_cubelist(yr, general_filename)

        # Join them into one (with error handling to deal with times which are wrong)
        try:
            full_year_cube = cubes.concatenate_cube()
            print("Concatenation successful!")
        except Exception as e:
            print(f"Initial concatenation failed: {str(e)}")

            # If initial concatenation fails, remove problematic cubes and try again
            try:
                full_year_cube = remove_problematic_cubes(cubes)
                print("Concatenation successful after removing problematic cubes!")
            except RuntimeError as e:
                print(f"Concatenation failed after removing problematic cubes: {str(e)}")               
        save_cube_as_pickle_file(full_year_cube, pickle_file_filepath)
        
        
    gauge_nums = range(0,1294)
    # Function to process each gauge
    for gauge_num in gauge_nums:
        if not gauge_num in [423, 444, 827, 888]:
                print(f"year is {yr}, gauge num is {gauge_num}")             

                base_dir = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/WholeYear"
                # Create the directory if it doesnt exist
                if not os.path.isdir(base_dir):
                    os.makedirs(base_dir)

                ######################################################
                ## Check if any files are missing, across the 3 filtering options
                # If there are: code will continue to run
                # If not: code will move to next gauge
                ######################################################                
                # Create a flag to record whether we are missing any of the files we need
                missing_files = False

                # Check if we are missing any of the files, and if so, change the flag to True
                if not any(os.path.exists(f"{base_dir}/{duration}hrs_{yr}_v2_part0.csv") for duration in [0.5, 1, 2, 3, 6, 12, 24]):
                    missing_files = True

                # If we are missing some files then get the data for the grid cell, 
                if missing_files:
                    # Find location
                    Tb0, idx_2d = find_gauge_Tb0_and_location_in_grid(tbo_vals, gauge_num, sample_cube)

                    # Extract data for the specified indices
                    start= time.time()
                    one_location_cube = full_year_cube[:, idx_2d[0], idx_2d[1]]
                    data = one_location_cube.data
                    end=time.time()
                    print(f"Time to load data is {round(end-start,2)} seconds")

                    ##### Filter cube according to different options
                    # Convert to dataframe
                    df = pd.DataFrame(data, columns=['precipitation (mm/hr)'])
                    df['times'] = one_location_cube.coord('time').units.num2date(one_location_cube.coord('time').points)
                    df['precipitation (mm)'] = df['precipitation (mm/hr)'] / 2   

                    # Search dataframe for events corresponding to durations
                    for duration in [0.5, 1, 2, 3, 6, 12, 24]:

                        filename =  f"{base_dir}/{duration}hrs_{yr}_v2_part0.csv"
                        if not os.path.exists(filename):
                            print(f"Finding the AMAX for {duration}hr events for gauge {gauge_num} in year {yr}")
                            # Find events
                            events_v2 = search_for_valid_events(df, duration=duration, Tb0=Tb0)

                            # Save events to CSV
                            for num, event in enumerate(events_v2):
                                if len(event) > 1:
                                        event.to_csv(f"{base_dir}/{duration}hrs_{yr}_v2_part{num}.csv")
                                        if event['precipitation (mm/hr)'].isna().any():
                                            print("NANs in this event")
                        else:
                            # print(f"already exists{filename}")
                            pass   
                else:
                    pass
                    # print("already exist")



2061
Pickle file exists, so loading that
gauge num is 0
already exist
gauge num is 1
already exist
gauge num is 2
already exist
gauge num is 3
already exist
gauge num is 4
already exist
gauge num is 5
already exist
gauge num is 6
already exist
gauge num is 7
already exist
gauge num is 8
already exist
gauge num is 9
already exist
gauge num is 10
already exist
gauge num is 11
already exist
gauge num is 12
already exist
gauge num is 13
already exist
gauge num is 14
already exist
gauge num is 15
already exist
gauge num is 16
already exist
gauge num is 17
already exist
gauge num is 18
already exist
gauge num is 19
already exist
gauge num is 20
already exist
gauge num is 21
already exist
gauge num is 22
already exist
gauge num is 23
already exist
gauge num is 24
already exist
gauge num is 25
already exist
gauge num is 26
already exist
gauge num is 27
already exist
gauge num is 28
already exist
gauge num is 29
already exist
gauge num is 30
already exist
gauge num is 31
already exist
gauge num

already exist
gauge num is 338
already exist
gauge num is 339
already exist
gauge num is 340
already exist
gauge num is 341
already exist
gauge num is 342
already exist
gauge num is 343
already exist
gauge num is 344
already exist
gauge num is 345
already exist
gauge num is 346
already exist
gauge num is 347
already exist
gauge num is 348
already exist
gauge num is 349
already exist
gauge num is 350
already exist
gauge num is 351
already exist
gauge num is 352
already exist
gauge num is 353
already exist
gauge num is 354
already exist
gauge num is 355
already exist
gauge num is 356
already exist
gauge num is 357
already exist
gauge num is 358
already exist
gauge num is 359
already exist
gauge num is 360
already exist
gauge num is 361
already exist
gauge num is 362
already exist
gauge num is 363
already exist
gauge num is 364
already exist
gauge num is 365
already exist
gauge num is 366
already exist
gauge num is 367
already exist
gauge num is 368
already exist
gauge num is 369
already 

already exist
gauge num is 629
already exist
gauge num is 630
already exist
gauge num is 631
already exist
gauge num is 632
already exist
gauge num is 633
already exist
gauge num is 634
already exist
gauge num is 635
already exist
gauge num is 636
already exist
gauge num is 637
already exist
gauge num is 638
already exist
gauge num is 639
already exist
gauge num is 640
already exist
gauge num is 641
already exist
gauge num is 642
already exist
gauge num is 643
already exist
gauge num is 644
already exist
gauge num is 645
already exist
gauge num is 646
already exist
gauge num is 647
already exist
gauge num is 648
already exist
gauge num is 649
already exist
gauge num is 650
already exist
gauge num is 651
already exist
gauge num is 652
already exist
gauge num is 653
already exist
gauge num is 654
already exist
gauge num is 655
already exist
gauge num is 656
already exist
gauge num is 657
already exist
gauge num is 658
already exist
gauge num is 659
already exist
gauge num is 660
already 

already exist
gauge num is 979
already exist
gauge num is 980
already exist
gauge num is 981
already exist
gauge num is 982
already exist
gauge num is 983
already exist
gauge num is 984
already exist
gauge num is 985
already exist
gauge num is 986
already exist
gauge num is 987
already exist
gauge num is 988
already exist
gauge num is 989
already exist
gauge num is 990
already exist
gauge num is 991
already exist
gauge num is 992
already exist
gauge num is 993
already exist
gauge num is 994
already exist
gauge num is 995
already exist
gauge num is 996
already exist
gauge num is 997
already exist
gauge num is 998
already exist
gauge num is 999
already exist
gauge num is 1000
already exist
gauge num is 1001
already exist
gauge num is 1002
already exist
gauge num is 1003
already exist
gauge num is 1004
already exist
gauge num is 1005
already exist
gauge num is 1006
already exist
gauge num is 1007
already exist
gauge num is 1008
already exist
gauge num is 1009
already exist
gauge num is 10

already exist
gauge num is 1266
already exist
gauge num is 1267
already exist
gauge num is 1268
already exist
gauge num is 1269
already exist
gauge num is 1270
already exist
gauge num is 1271
already exist
gauge num is 1272
already exist
gauge num is 1273
already exist
gauge num is 1274
already exist
gauge num is 1275
already exist
gauge num is 1276
already exist
gauge num is 1277
already exist
gauge num is 1278
already exist
gauge num is 1279
already exist
gauge num is 1280
already exist
gauge num is 1281
already exist
gauge num is 1282
already exist
gauge num is 1283
already exist
gauge num is 1284
already exist
gauge num is 1285
already exist
gauge num is 1286
already exist
gauge num is 1287
already exist
gauge num is 1288
already exist
gauge num is 1289
already exist
gauge num is 1290
already exist
gauge num is 1291
already exist
gauge num is 1292
already exist
gauge num is 1293
already exist
2062
Pickle file exists, so loading that
gauge num is 0
already exist
gauge num is 1
alrea

...

already exist
gauge num is 688
already exist
gauge num is 689
already exist
gauge num is 690
already exist
gauge num is 691
already exist
gauge num is 692
already exist
gauge num is 693
already exist
gauge num is 694
already exist
gauge num is 695
already exist
gauge num is 696
already exist
gauge num is 697
already exist
gauge num is 698
already exist
gauge num is 699
already exist
gauge num is 700
already exist
gauge num is 701
already exist
gauge num is 702
already exist
gauge num is 703
already exist
gauge num is 704
already exist
gauge num is 705
already exist
gauge num is 706
already exist
gauge num is 707
already exist
gauge num is 708
already exist
gauge num is 709
already exist
gauge num is 710
already exist
gauge num is 711
already exist
gauge num is 712
already exist
gauge num is 713
already exist
gauge num is 714
already exist
gauge num is 715
already exist
gauge num is 716
already exist
gauge num is 717
already exist
gauge num is 718
already exist
gauge num is 719
already 

already exist
gauge num is 1181
already exist
gauge num is 1182
already exist
gauge num is 1183
already exist
gauge num is 1184
already exist
gauge num is 1185
already exist
gauge num is 1186
already exist
gauge num is 1187
already exist
gauge num is 1188
already exist
gauge num is 1189
already exist
gauge num is 1190
already exist
gauge num is 1191
already exist
gauge num is 1192
already exist
gauge num is 1193
already exist
gauge num is 1194
already exist
gauge num is 1195
already exist
gauge num is 1196
already exist
gauge num is 1197
already exist
gauge num is 1198
already exist
gauge num is 1199
already exist
gauge num is 1200
already exist
gauge num is 1201
already exist
gauge num is 1202
already exist
gauge num is 1203
already exist
gauge num is 1204
already exist
gauge num is 1205
already exist
gauge num is 1206
already exist
gauge num is 1207
already exist
gauge num is 1208
already exist
gauge num is 1209
already exist
gauge num is 1210
already exist
gauge num is 1211
already 

already exist
gauge num is 380
already exist
gauge num is 381
already exist
gauge num is 382
already exist
gauge num is 383
already exist
gauge num is 384
already exist
gauge num is 385
already exist
gauge num is 386
already exist
gauge num is 387
already exist
gauge num is 388
already exist
gauge num is 389
already exist
gauge num is 390
already exist
gauge num is 391
already exist
gauge num is 392
already exist
gauge num is 393
already exist
gauge num is 394
already exist
gauge num is 395
already exist
gauge num is 396
already exist
gauge num is 397
already exist
gauge num is 398
already exist
gauge num is 399
already exist
gauge num is 400
already exist
gauge num is 401
already exist
gauge num is 402
already exist
gauge num is 403
already exist
gauge num is 404
already exist
gauge num is 405
already exist
gauge num is 406
already exist
gauge num is 407
already exist
gauge num is 408
already exist
gauge num is 409
already exist
gauge num is 410
already exist
gauge num is 411
already 

already exist
gauge num is 900
already exist
gauge num is 901
already exist
gauge num is 902
already exist
gauge num is 903
already exist
gauge num is 904
already exist
gauge num is 905
already exist
gauge num is 906
already exist
gauge num is 907
already exist
gauge num is 908
already exist
gauge num is 909
already exist
gauge num is 910
already exist
gauge num is 911
already exist
gauge num is 912
already exist
gauge num is 913
already exist
gauge num is 914
already exist
gauge num is 915
already exist
gauge num is 916
already exist
gauge num is 917
already exist
gauge num is 918
already exist
gauge num is 919
already exist
gauge num is 920
already exist
gauge num is 921
already exist
gauge num is 922
already exist
gauge num is 923
already exist
gauge num is 924
already exist
gauge num is 925
already exist
gauge num is 926
already exist
gauge num is 927
already exist
gauge num is 928
already exist
gauge num is 929
already exist
gauge num is 930
already exist
gauge num is 931
already 

(228, 297)
Time to load data is 303.25 seconds
Finding the AMAX for 0.5hr events for gauge 41 in year 2072
Event doesnt contain NAN, total event precip is 47.83373260498047
Finding the AMAX for 1hr events for gauge 41 in year 2072
Event doesnt contain NAN, total event precip is 47.83373260498047
Finding the AMAX for 2hr events for gauge 41 in year 2072
Event doesnt contain NAN, total event precip is 47.83373260498047
Finding the AMAX for 3hr events for gauge 41 in year 2072
Event doesnt contain NAN, total event precip is 47.83373260498047
Finding the AMAX for 6hr events for gauge 41 in year 2072
Event doesnt contain NAN, total event precip is 47.834774017333984
Finding the AMAX for 12hr events for gauge 41 in year 2072
Event doesnt contain NAN, total event precip is 103.07403564453125
Finding the AMAX for 24hr events for gauge 41 in year 2072
Event doesnt contain NAN, total event precip is 103.16488647460938
gauge num is 42
(230, 316)
Time to load data is 42.0 seconds
Finding the AMAX 